# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`]
- Define feature and target variables X and Y

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import warnings
warnings.filterwarnings("ignore")
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\InternetUser\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\InternetUser\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\InternetUser\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
#engine = create_engine('sqlite:///data\\disaster_repsonse.db')

In [3]:
# load data from database
engine = create_engine('sqlite:///data//DisasterResponse.db')
df = pd.read_sql('clean_data',con=engine)

X = df['message'].values
Y = df[df.columns[4:]]

In [4]:
# display the value of x
X

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'],
      dtype=object)

In [5]:
# display the various columns
df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [6]:
# display the y values
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [7]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [8]:
def replace_urls(text):
    """
    Outputs editted version of the input Python str object `text` 
    replacing all urls in text with str 'urlplaceholder'.
    
    INPUT:
        - text - Python str object - a raw text data
        
    OUTPUT:
        - text - Python str object - An editted version of the input data `text` 
          with all urls in text replacing with str 'urlplaceholder'.
    """
    
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')
        
    return text

In [9]:
def tokenize(text):
    """
      Takes a Python string object and outputs list of processed words 
       of the text.
      INPUT:
        - text - Python str object - A raw text data
      OUTPUT:
        - stem_words - Python list object - list of processed words using the input `text`.
     """
    #function with raw text data    
    text = replace_urls(text)
    # Removes punctuations and covert to lower case 
    #text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text).lower() 
    text = word_tokenize(text) 
    # Remove stop words
    text = [w for w in text if w not in stopwords.words("english")]
    # Lemmatize verbs by specifying pos
    text = [WordNetLemmatizer().lemmatize(w) for w in text]
    
    return text


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
# create the pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs=-1)))
])

In [11]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x0000019B71D92820>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000019B71D92820>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__subl

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [13]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000019B71D92820>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))])

In [14]:
y_pred = pipeline.predict(X_test)

In [15]:
y_pred.shape,y_test.shape,len(list(Y.columns))

((6554, 36), (6554, 36), 36)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
print(classification_report(y_test,y_pred,target_names = df.columns[4:]))

                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      5018
               request       0.83      0.49      0.62      1132
                 offer       0.00      0.00      0.00        30
           aid_related       0.76      0.69      0.72      2763
          medical_help       0.69      0.09      0.16       509
      medical_products       0.87      0.08      0.14       348
     search_and_rescue       0.33      0.02      0.03       199
              security       0.50      0.01      0.02       101
              military       0.77      0.05      0.09       213
           child_alone       0.00      0.00      0.00         0
                 water       0.89      0.36      0.51       418
                  food       0.83      0.61      0.71       744
               shelter       0.83      0.34      0.48       569
              clothing       0.71      0.11      0.18        95
                 money       0.80      

### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
parameters = {
        'clf__estimator__n_estimators': [100,150,250]
        }

In [18]:
cv = GridSearchCV(pipeline, param_grid=parameters)

In [19]:
best_model = cv.fit(X_train,y_train)

In [20]:
y_pred = best_model.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [21]:
print(classification_report(y_test,y_pred,target_names = df.columns[4:]))

                        precision    recall  f1-score   support

               related       0.83      0.95      0.89      5018
               request       0.83      0.50      0.62      1132
                 offer       0.00      0.00      0.00        30
           aid_related       0.77      0.70      0.73      2763
          medical_help       0.71      0.09      0.17       509
      medical_products       0.89      0.09      0.16       348
     search_and_rescue       0.50      0.01      0.02       199
              security       0.50      0.01      0.02       101
              military       0.86      0.06      0.11       213
           child_alone       0.00      0.00      0.00         0
                 water       0.89      0.39      0.54       418
                  food       0.84      0.64      0.73       744
               shelter       0.83      0.38      0.52       569
              clothing       0.77      0.11      0.19        95
                 money       0.75      

### 8. Export your model as a pickle file

In [22]:
# save the model to disk
filename = 'random_forest_classifier_model.pkl'
pickle.dump(pipeline, open(filename, 'wb'))

### 9. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.